In [2]:
import numpy as np
import cv2
from cv2 import aruco
import sys
import time
import csv
import datetime
import matplotlib.pyplot as plt
import concurrent.futures
from threading import Thread

tmp0=[0,0,0,0,0,0,0]
tmp1=[0,0,0,0,0,0,0]
tmp2=[0,0,0,0,0,0,0]
tmp3=[0,0,0,0,0,0,0]
tmp4=[0,0,0,0,0,0,0]
tmp5=[0,0,0,0,0,0,0]

def d_e(num,image, cameraMatrix, distCoeffs,tmp):
    arucoDict = aruco.Dictionary_get(aruco.DICT_4X4_50)
    parameters = aruco.DetectorParameters_create()
    corners0, ids0, rejectedImgPoints0 = aruco.detectMarkers(image, arucoDict, parameters = parameters)
    rvecs0, tvecs0, _objPoints0 = aruco.estimatePoseSingleMarkers(corners0, 0.06, cameraMatrix, distCoeffs)
    
    tmp[0] = corners0
    tmp[1] = ids0
    tmp[2] = rejectedImgPoints0
    tmp[3] = rvecs0
    tmp[4] = tvecs0
    tmp[5] = _objPoints0
    


if __name__ == '__main__':
    abs_coor={
        0:(-90,90), 1:(-60,90), 2:(-30,90), 3:(0,90), 4:(30,90), 5:(60,90), 6:(90,90), 
        7:(-90,60), 8:(-60,60), 9:(-30,60), 10:(0,60), 11:(30,60), 12:(60,60), 13:(90,60),
        14:(-90,30), 15:(-60,30), 16:(-30,30), 17:(0,30), 18:(30,30), 19:(60,30), 20:(90,30),
        21:(-90,0), 22:(-60,0), 23:(-30,0), 24:(0,0), 25:(30,0), 26:(60,0), 27:(90,0),
        28:(-90,-30), 29:(-60,-30), 30:(-30,-30), 31:(0,-30), 32:(30,-30), 33:(60,-30), 34:(90,-30),
        35:(-90,-60), 36:(-60,-60), 37:(-30,-60), 38:(0,-60), 39:(30,-60), 40:(60,-60), 41:(90,-60),
        42:(-90,-90), 43:(-60,-90), 44:(-30,-90), 45:(0,-90), 46:(30,-90), 47:(60,-90), 48:(90,-90)
    }
    id_cnt = [0 for _ in range(49)]
    

    #camera setting
    cap0 = cv2.VideoCapture(cv2.CAP_DSHOW+0) #아래
    cap1 = cv2.VideoCapture(cv2.CAP_DSHOW+1) #중간
    cap2 = cv2.VideoCapture(cv2.CAP_DSHOW+2) #위
    cap3 = cv2.VideoCapture(cv2.CAP_DSHOW+3) #위
    cap4 = cv2.VideoCapture(cv2.CAP_DSHOW+4) #위
    cap5 = cv2.VideoCapture(cv2.CAP_DSHOW+5) #위

    cap0.set(cv2.CAP_PROP_AUTOFOCUS, 0)
    cap1.set(cv2.CAP_PROP_AUTOFOCUS, 0)
    cap2.set(cv2.CAP_PROP_AUTOFOCUS, 0)
    cap3.set(cv2.CAP_PROP_AUTOFOCUS, 0)
    cap4.set(cv2.CAP_PROP_AUTOFOCUS, 0)
    cap5.set(cv2.CAP_PROP_AUTOFOCUS, 0)

    #초점 조절 (parameter, 초점값 (0~250))
    cap0.set(28, 0)
    cap1.set(28, 0)
    cap2.set(28, 0)
    cap3.set(28, 0)
    cap4.set(28, 0)
    cap5.set(28, 0)


    cap0.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap0.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    cap1.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap1.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    cap2.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap2.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    cap3.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap3.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    cap4.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap4.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    cap5.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap5.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    CAM0_X = 131
    CAM0_Y = 21
    CAM1_X = 133
    CAM1_Y = 91
    CAM2_X = -15.5
    CAM2_Y = -36
    CAM3_X = -17
    CAM3_Y = 22.5
    CAM4_X = 127
    CAM4_Y = -62
    CAM5_X = -16
    CAM5_Y = 94

    CAM0_PX = 90
    CAM0_PY = 0
    CAM1_PX = 90
    CAM1_PY = 90
    CAM2_PX = -90
    CAM2_PY = -90
    CAM3_PX = -90
    CAM3_PY = 0
    CAM4_PX = 90
    CAM4_PY = -90
    CAM5_PX = -90
    CAM5_PY = 90

    BIN_CONST = 110

    cameraMatrix = np.array([[1701.372,0,952.141],[0,1713.938,  473.713],[0,0,1]])
    distCoeffs = (-0.519923,  1.739463,  0.052515,  -0.040351)


    cameraMatrix2 = np.array([[1413.041,0,961.264],[0,1446.267, 611.979],[0,0,1]])
    distCoeffs2 = (-0.805243,  2.580690,  -0.000427,  -0.012968)

    arucoDict = aruco.Dictionary_get(aruco.DICT_4X4_50)
    parameters = aruco.DetectorParameters_create()
    rotM = np.zeros(shape=(3,3))
    
    while(True):


        ret0,image0 = cap0.read()
        ret1,image1 = cap1.read()
        ret2,image2 = cap2.read()
        ret3,image3 = cap3.read()
        ret4,image4 = cap4.read()
        ret5,image5 = cap5.read()


        t0 = Thread(target=d_e, args=(0,image0, cameraMatrix, distCoeffs,tmp0))
        t1 = Thread(target=d_e, args=(1,image1, cameraMatrix, distCoeffs,tmp1))
        t2 = Thread(target=d_e, args=(2,image2, cameraMatrix2, distCoeffs2,tmp2))
        t3 = Thread(target=d_e, args=(3,image3, cameraMatrix, distCoeffs,tmp3))
        t4 = Thread(target=d_e, args=(4,image4, cameraMatrix, distCoeffs,tmp4))
        t5 = Thread(target=d_e, args=(5,image5, cameraMatrix, distCoeffs,tmp5))
        
        t0.start()
        t1.start()
        t2.start()
        t3.start()
        t4.start()
        t5.start()
        
        t0.join()
        t1.join()
        t2.join()
        t3.join()
        t4.join()
        t5.join()
        
        corners0 = tmp0[0]
        ids0 = tmp0[1]
        rejectedImgPoints0 = tmp0[2]
        rvecs0 = tmp0[3]
        tvecs0 = tmp0[4]
        _objPoints0 = tmp0[5]
        
        
        corners1 = tmp1[0]
        ids1 = tmp1[1]
        rejectedImgPoint1 = tmp1[2]
        rvecs1 = tmp1[3]
        tvecs1 = tmp1[4]
        _objPoints1 = tmp1[5]
        
        
        corners2 = tmp2[0]
        ids2 = tmp2[1]
        rejectedImgPoints2 = tmp2[2]
        rvecs2 = tmp2[3]
        tvecs2 = tmp2[4]
        _objPoints2 = tmp2[5]
        
        
        corners3 = tmp3[0]
        ids3 = tmp3[1]
        rejectedImgPoints3 = tmp3[2]
        rvecs3 = tmp3[3]
        tvecs3 = tmp3[4]
        _objPoints3 = tmp3[5]
        
        
        corners4 = tmp4[0]
        ids4 = tmp4[1]
        rejectedImgPoints4 = tmp4[2]
        rvecs4 = tmp4[3]
        tvecs4 = tmp4[4]
        _objPoints4 = tmp4[5]
        
        
        corners5 = tmp5[0]
        ids5 = tmp5[1]
        rejectedImgPoints5 = tmp5[2]
        rvecs5 = tmp5[3]
        tvecs5 = tmp5[4]
        _objPoints5 = tmp5[5]
        

        al = {0:[],1:[],2:[],3:[],4:[],5:[],6:[],
              7:[],8:[],9:[],10:[],11:[],12:[],13:[],
              14:[],15:[],16:[],17:[],18:[],19:[],20:[],
              21:[],22:[],23:[],24:[],25:[],26:[],27:[],
              28:[],29:[],30:[],31:[],32:[],33:[],34:[],
              35:[],36:[],37:[],38:[],39:[],40:[],41:[],
              42:[],43:[],44:[],45:[],46:[],47:[],48:[]}

        try:
            ids_list = []
            for i in ids0:
                ids_list.append(i[0])
            for i in ids1:
                ids_list.append(i[0])
            for i in ids2:
                ids_list.append(i[0])
            for i in ids3:
                ids_list.append(i[0])
            for i in ids4:
                ids_list.append(i[0])
            for i in ids5:
                ids_list.append(i[0])
            np.unique(ids_list)
        except TypeError:
            print(TypeError)
            


        
        for i in ids_list:
            # 카메라 한대에서만 탐지된 경우 section 0 ~ 5
            #section 0 
            if i in ids2 and i not in ids0 and i not in ids1 and i not in ids3 and i not in ids4 and i not in ids5:
                #print("section 0 :",i)
                target = np.where(ids2 == i)
                target = target[0][0]
                aruco.drawAxis(image2, cameraMatrix2, distCoeffs2, rvecs2[target], tvecs2[target], 0.06)
                cv2.Rodrigues(rvecs2[target], rotM, jacobian = 0)
                ypr = cv2.RQDecomp3x3(rotM)

                #idr is robot's id
                idr = i
                x = tvecs2[target][0][0] * 100 + CAM2_X
                y = tvecs2[target][0][1] * 100 + CAM2_Y
                
                seta = ypr[0][2]

                al[idr]=[x,y,seta]

                #print(idr,x,y,seta)

            #section 1
            elif i in ids4 and i not in ids0 and i not in ids1 and i not in ids3 and i not in ids2 and i not in ids5:
                #print("section 1 :",i)
                target = np.where(ids4 == i)
                target = target[0][0]
                aruco.drawAxis(image4, cameraMatrix, distCoeffs, rvecs4[target], tvecs4[target], 0.06)
                cv2.Rodrigues(rvecs4[target], rotM, jacobian = 0)
                ypr = cv2.RQDecomp3x3(rotM)

                #idr is robot's id
                idr = i
                x = tvecs4[target][0][0] * 100 + CAM4_X
                y = tvecs4[target][0][1] * 100 + CAM4_Y
                
                seta = ypr[0][2]

                al[idr]=[x,y,seta]
                #print(idr,x,y,seta)

            #section 2
            elif i in ids3 and i not in ids0 and i not in ids1 and i not in ids4 and i not in ids2 and i not in ids5:
                #print("section 2 :",i)
                target = np.where(ids3 == i)
                target = target[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target], tvecs3[target], 0.06)
                cv2.Rodrigues(rvecs3[target], rotM, jacobian = 0)
                ypr = cv2.RQDecomp3x3(rotM)

                #idr is robot's id
                idr = i
                x = tvecs3[target][0][0] * 100 + CAM3_X
                y = tvecs3[target][0][1] * 100 + CAM3_Y
                
                seta = ypr[0][2]

                al[idr]=[x,y,seta]
                #print(idr,x,y,seta)

            #section 3
            elif i in ids0 and i not in ids3 and i not in ids1 and i not in ids4 and i not in ids2 and i not in ids5:
                #print("section 3 :",i)
                target = np.where(ids0 == i)
                target = target[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target], tvecs0[target], 0.06)
                cv2.Rodrigues(rvecs0[target], rotM, jacobian = 0)
                ypr = cv2.RQDecomp3x3(rotM)

                #idr is robot's id
                idr = i
                x = tvecs0[target][0][0] * 100 + CAM0_X
                y = tvecs0[target][0][1] * 100 + CAM0_Y
                
                seta = ypr[0][2]

                al[idr]=[x,y,seta]
                #print(idr,x,y,seta)

            #section 4
            elif i in ids5 and i not in ids3 and i not in ids1 and i not in ids4 and i not in ids2 and i not in ids0:
                #print("section 4 :",i)
                target = np.where(ids5 == i)
                target = target[0][0]
                aruco.drawAxis(image5, cameraMatrix, distCoeffs, rvecs5[target], tvecs5[target], 0.06)
                cv2.Rodrigues(rvecs5[target], rotM, jacobian = 0)
                ypr = cv2.RQDecomp3x3(rotM)

                #idr is robot's id
                idr = i
                x = tvecs5[target][0][0] * 100 + CAM5_X
                y = tvecs5[target][0][1] * 100 + CAM5_Y
                
                seta = ypr[0][2]

                al[idr]=[x,y,seta]
                #print(idr,x,y,seta)

            #section 5
            elif i in ids1 and i not in ids3 and i not in ids0 and i not in ids4 and i not in ids2 and i not in ids5:
                #print("section 5 :",i)

                target = np.where(ids1 == i)
                target = target[0][0]
                aruco.drawAxis(image1, cameraMatrix, distCoeffs, rvecs1[target], tvecs1[target], 0.06)
                cv2.Rodrigues(rvecs1[target], rotM, jacobian = 0)
                ypr = cv2.RQDecomp3x3(rotM)

                #idr is robot's id
                idr = i
                x = tvecs1[target][0][0] * 100 + CAM1_X
                y = tvecs1[target][0][1] * 100 + CAM1_Y
                
                seta = ypr[0][2]

                al[idr]=[x,y,seta]
                #print(idr,x,y,seta)

            # 카메라 두대에서 탐지된 경우 section 6 ~ 12
            #section 6
            elif i in ids2 and i in ids4 and i not in ids0 and i not in ids1 and i not in ids3 and i not in ids5:
                #print("section 6 :",i)
                target2 = np.where(ids2 == i)
                target2 = target2[0][0]


                aruco.drawAxis(image2, cameraMatrix2, distCoeffs2, rvecs2[target2], tvecs2[target2], 0.06 )

                cv2.Rodrigues(rvecs2[target2], rotM, jacobian = 0)
                ypr2 = cv2.RQDecomp3x3(rotM)
                idr = i

                x2=tvecs2[target2][0][0] * 100 + CAM2_X
                y2=tvecs2[target2][0][1] * 100 + CAM2_Y
                seta2 = ypr2[0][2]

                xc2 = (corners2[target2][0][0][0] + corners2[target2][0][1][0] + corners2[target2][0][2][0] + corners2[target2][0][3][0]) / 4
                yc2 = (corners2[target2][0][0][1] + corners2[target2][0][1][1] + corners2[target2][0][2][1] + corners2[target2][0][3][1]) / 4

                d2 = (abs(xc2-CAM2_PX)**2+abs(yc2-CAM2_PY)**2)**0.5
                

                target4 = np.where(ids4 == i)
                target4 = target4[0][0]
                aruco.drawAxis(image4, cameraMatrix, distCoeffs, rvecs4[target4], tvecs4[target4], 0.06 )
                cv2.Rodrigues(rvecs4[target4], rotM, jacobian = 0)
                ypr4 = cv2.RQDecomp3x3(rotM)

                x4=tvecs4[target4][0][0] * 100 + CAM4_X
                y4=tvecs4[target4][0][1] * 100 + CAM4_Y
                seta4 = ypr4[0][2]

                xc4 = (corners4[target4][0][0][0] + corners4[target4][0][1][0] + corners4[target4][0][2][0] + corners4[target4][0][3][0]) / 4
                yc4 = (corners4[target4][0][0][1] + corners4[target4][0][1][1] + corners4[target4][0][2][1] + corners4[target4][0][3][1]) / 4

                d4 = (abs(xc4-CAM4_PX)**2+abs(yc4-CAM4_PY)**2)**0.5
                
                w = d2/(d4+d2)

                x_f = (w*x2 + (1-w)*x4)
                y_f = (w*y2 + (1-w)*y4)
                seta = w*seta2 + (1-w)*seta4

                al[idr]=[x_f,y_f,seta]
                #print(idr, x_f,y_f,x2,x4,y2,y4,w)


            #section 7
            elif i in ids2 and i in ids3 and i not in ids0 and i not in ids1 and i not in ids4 and i not in ids5:
                #print("section 7 :",i)
                target2 = np.where(ids2 == i)
                target2 = target2[0][0]


                aruco.drawAxis(image2, cameraMatrix2, distCoeffs2, rvecs2[target2], tvecs2[target2], 0.06 )

                cv2.Rodrigues(rvecs2[target2], rotM, jacobian = 0)
                ypr2 = cv2.RQDecomp3x3(rotM)
                idr = i

                x2=tvecs2[target2][0][0] * 100 + CAM2_X
                y2=tvecs2[target2][0][1] * 100 + CAM2_Y
                seta2 = ypr2[0][2]

                xc2 = (corners2[target2][0][0][0] + corners2[target2][0][1][0] + corners2[target2][0][2][0] + corners2[target2][0][3][0]) / 4
                yc2 = (corners2[target2][0][0][1] + corners2[target2][0][1][1] + corners2[target2][0][2][1] + corners2[target2][0][3][1]) / 4

                d2 = (abs(xc2-CAM2_PX)**2+abs(yc2-CAM2_PY)**2)**0.5
                

                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM)

                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]

                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4

                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                
                w = d2/(d2+d3)

                x_f = (w*x2 + (1-w)*x3)
                y_f = (w*y2 + (1-w)*y3)
                seta = w*seta2 + (1-w)*seta3
                
                al[idr]=[x_f,y_f,seta]
                #print(idr, x_f,y_f,x2,x3,y2,y3,w)

            #section 8
            elif i in ids0 and i in ids4 and i not in ids1 and i not in ids2 and i not in ids3 and i not in ids5:
                #print("section 8 :",i)
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]


                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )

                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i

                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]

                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4

                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5
                

                target4 = np.where(ids4 == i)
                target4 = target4[0][0]
                aruco.drawAxis(image4, cameraMatrix, distCoeffs, rvecs4[target4], tvecs4[target4], 0.06 )
                cv2.Rodrigues(rvecs4[target4], rotM, jacobian = 0)
                ypr4 = cv2.RQDecomp3x3(rotM)

                x4=tvecs4[target4][0][0] * 100 + CAM4_X
                y4=tvecs4[target4][0][1] * 100 + CAM4_Y
                seta4 = ypr4[0][2]

                xc4 = (corners4[target4][0][0][0] + corners4[target4][0][1][0] + corners4[target4][0][2][0] + corners4[target4][0][3][0]) / 4
                yc4 = (corners4[target4][0][0][1] + corners4[target4][0][1][1] + corners4[target4][0][2][1] + corners4[target4][0][3][1]) / 4

                d4 = (abs(xc4-CAM4_PX)**2+abs(yc4-CAM4_PY)**2)**0.5
                
                w = d0/(d0+d4)

                x_f = (w*x0 + (1-w)*x4)
                y_f = (w*y0 + (1-w)*y4)
                seta = w*seta0 + (1-w)*seta4

                al[idr]=[x_f,y_f,seta]
                #print(idr, x_f,y_f,x0,x4,y0,y4,w)

            #section 9
            elif i in ids0 and i in ids3 and i not in ids1 and i not in ids2 and i not in ids4 and i not in ids5:
                #print("section 9 :",i)
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]


                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )

                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i

                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]

                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4

                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5
                

                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM)

                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]

                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4

                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                
                w = d0/(d0+d3)

                x_f = (w*x0 + (1-w)*x3)
                y_f = (w*y0 + (1-w)*y3)
                seta = w*seta0 + (1-w)*seta3

                al[idr]=[x_f,y_f,seta]
                #print(idr, x_f,y_f,x0,x3,y0,y3,w)


            #section 10
            elif i in ids3 and i in ids5 and i not in ids0 and i not in ids1 and i not in ids2 and i not in ids4:
                #print("section 10 :",i)
                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM)

                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]

                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4

                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                
                w = d2/(d2+d3)

                target5 = np.where(ids5 == i)
                target5 = target5[0][0]
                aruco.drawAxis(image5, cameraMatrix, distCoeffs, rvecs5[target5], tvecs5[target5], 0.06)
                cv2.Rodrigues(rvecs5[target5], rotM, jacobian = 0)
                ypr5 = cv2.RQDecomp3x3(rotM)

                #idr is robot's id
                idr = i
                x5 = tvecs5[target5][0][0] * 100 + CAM5_X
                y5 = tvecs5[target5][0][1] * 100 + CAM5_Y

                seta5 = ypr5[0][2]

                xc5 = (corners5[target5][0][0][0] + corners5[target5][0][1][0] + corners5[target5][0][2][0] + corners5[target5][0][3][0]) / 4
                yc5 = (corners5[target5][0][0][1] + corners5[target5][0][1][1] + corners5[target5][0][2][1] + corners5[target5][0][3][1]) / 4

                d5 = (abs(xc5-CAM5_PX)**2+abs(yc5-CAM5_PY)**2)**0.5
                
                w = d3/(d3+d5)

                x_f = (w*x3 + (1-w)*x5)
                y_f = (w*y3 + (1-w)*y5)
                seta = w*seta3 + (1-w)*seta5

                al[idr]=[x_f,y_f,seta]
                #print(idr, x_f,y_f,x3,x5,y3,y5,w)


            #section 11
            elif i in ids0 and i in ids1 and i not in ids2 and i not in ids3 and i not in ids4 and i not in ids5:
                #print("section 11 :",i)
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]


                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )

                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i

                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]

                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4

                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5
                

                target1 = np.where(ids1 == i)
                target1 = target1[0][0]


                aruco.drawAxis(image1, cameraMatrix, distCoeffs, rvecs1[target1], tvecs1[target1], 0.06 )

                cv2.Rodrigues(rvecs1[target1], rotM, jacobian = 0)
                ypr1 = cv2.RQDecomp3x3(rotM)
                idr = i

                x1=tvecs1[target1][0][0] * 100 + CAM1_X
                y1=tvecs1[target1][0][1] * 100 + CAM1_Y
                seta1 = ypr1[0][2]

                xc1 = (corners1[target1][0][0][0] + corners1[target1][0][1][0] + corners1[target1][0][2][0] + corners1[target1][0][3][0]) / 4
                yc1 = (corners1[target1][0][0][1] + corners1[target1][0][1][1] + corners1[target1][0][2][1] + corners1[target1][0][3][1]) / 4

                d1 = (abs(xc1-CAM1_PX)**2+abs(yc1-CAM1_PY)**2)**0.5
                

                w = d0/(d0+d1)

                x_f = (w*x0 + (1-w)*x1)
                y_f = (w*y0 + (1-w)*y1)
                seta = w*seta0 + (1-w)*seta1
                
                al[idr]=[x_f,y_f,seta]
                

            #section 12
            elif i in ids1 and i in ids5 and i not in ids0 and i not in ids2 and i not in ids3 and i not in ids4:
                #print("section 12 :",i)
                target1 = np.where(ids1 == i)
                target1 = target1[0][0]


                aruco.drawAxis(image1, cameraMatrix, distCoeffs, rvecs1[target1], tvecs1[target1], 0.06 )

                cv2.Rodrigues(rvecs1[target1], rotM, jacobian = 0)
                ypr1 = cv2.RQDecomp3x3(rotM)
                idr = i

                x1=tvecs1[target1][0][0] * 100 + CAM1_X
                y1=tvecs1[target1][0][1] * 100 + CAM1_Y
                seta1 = ypr1[0][2]

                xc1 = (corners1[target1][0][0][0] + corners1[target1][0][1][0] + corners1[target1][0][2][0] + corners1[target1][0][3][0]) / 4
                yc1 = (corners1[target1][0][0][1] + corners1[target1][0][1][1] + corners1[target1][0][2][1] + corners1[target1][0][3][1]) / 4

                d1 = (abs(xc1-CAM1_PX)**2+abs(yc1-CAM1_PY)**2)**0.5
                

                target5 = np.where(ids5 == i)
                target5 = target5[0][0]
                aruco.drawAxis(image5, cameraMatrix, distCoeffs, rvecs5[target5], tvecs5[target5], 0.06)
                cv2.Rodrigues(rvecs5[target5], rotM, jacobian = 0)
                ypr5 = cv2.RQDecomp3x3(rotM)

                #idr is robot's id
                idr = i
                x5 = tvecs5[target5][0][0] * 100 + CAM5_X
                y5 = tvecs5[target5][0][1] * 100 + CAM5_Y

                seta5 = ypr5[0][2]

                xc5 = (corners5[target5][0][0][0] + corners5[target5][0][1][0] + corners5[target5][0][2][0] + corners5[target5][0][3][0]) / 4
                yc5 = (corners5[target5][0][0][1] + corners5[target5][0][1][1] + corners5[target5][0][2][1] + corners5[target5][0][3][1]) / 4

                d5 = (abs(xc5-CAM5_PX)**2+abs(yc5-CAM5_PY)**2)**0.5
                

                w = d1/(d1+d5)

                x_f = (w*x1 + (1-w)*x5)
                y_f = (w*y1 + (1-w)*y5)
                seta = w*seta1 + (1-w)*seta5

                al[idr]=[x_f,y_f,seta]
                #print(idr, x_f,y_f,x1,x5,y1,y5,w)



            #카메라 네대에서 탐지된 경우 section 13 ~ 14
            #section 13
            elif i in ids0 and i in ids2 and i in ids3 and i in ids4 and i not in ids1 and i not in ids5:
                #print("section 13 :",i)
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )
                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i
                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]
                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4
                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5
                


                target2 = np.where(ids2 == i)
                target2 = target2[0][0]
                aruco.drawAxis(image2, cameraMatrix2, distCoeffs2, rvecs2[target2], tvecs2[target2], 0.06 )
                cv2.Rodrigues(rvecs2[target2], rotM, jacobian = 0)
                ypr2 = cv2.RQDecomp3x3(rotM)
                idr = i
                x2=tvecs2[target2][0][0] * 100 + CAM2_X
                y2=tvecs2[target2][0][1] * 100 + CAM2_Y
                seta2 = ypr2[0][2]
                xc2 = (corners2[target2][0][0][0] + corners2[target2][0][1][0] + corners2[target2][0][2][0] + corners2[target2][0][3][0]) / 4
                yc2 = (corners2[target2][0][0][1] + corners2[target2][0][1][1] + corners2[target2][0][2][1] + corners2[target2][0][3][1]) / 4
                d2 = (abs(xc2-CAM2_PX)**2+abs(yc2-CAM2_PY)**2)**0.5
                


                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM) 
                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]
                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4
                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                


                target4 = np.where(ids4 == i)
                target4 = target4[0][0]
                aruco.drawAxis(image4, cameraMatrix, distCoeffs, rvecs4[target4], tvecs4[target4], 0.06 )
                cv2.Rodrigues(rvecs4[target4], rotM, jacobian = 0)
                ypr4 = cv2.RQDecomp3x3(rotM)
                x4=tvecs4[target4][0][0] * 100 + CAM4_X
                y4=tvecs4[target4][0][1] * 100 + CAM4_Y
                seta4 = ypr4[0][2]
                xc4 = (corners4[target4][0][0][0] + corners4[target4][0][1][0] + corners4[target4][0][2][0] + corners4[target4][0][3][0]) / 4
                yc4 = (corners4[target4][0][0][1] + corners4[target4][0][1][1] + corners4[target4][0][2][1] + corners4[target4][0][3][1]) / 4
                d4 = (abs(xc4-CAM4_PX)**2+abs(yc4-CAM4_PY)**2)**0.5
                

                w0 = d0/(d0+d2+d3+d4)
                w2 = d2/(d0+d2+d3+d4)
                w3 = d3/(d0+d2+d3+d4)
                w4 = d4/(d0+d2+d3+d4)

                x_f = w0*x0 + w2*x2 + w3*x3 + w4*x4
                y_f = w0*y0 + w2*y2 + w3*y3 + w4*y4
                seta = w0*seta0 + w2*seta2 + w3*seta3 + w4*seta4
                #print(idr, x_f,y_f)
                al[idr]=[x_f,y_f,seta]


            #section 14
            elif i in ids0 and i in ids1 and i in ids3 and i in ids5 and i not in ids2 and i not in ids4:
                #print("section 14 :",i)
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )
                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i
                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]
                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4
                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5
                


                #print("section 12 :",i)
                target1 = np.where(ids1 == i)
                target1 = target1[0][0]
                aruco.drawAxis(image1, cameraMatrix, distCoeffs, rvecs1[target1], tvecs1[target1], 0.06 )
                cv2.Rodrigues(rvecs1[target1], rotM, jacobian = 0)
                ypr1 = cv2.RQDecomp3x3(rotM)
                idr = i
                x1=tvecs1[target1][0][0] * 100 + CAM1_X
                y1=tvecs1[target1][0][1] * 100 + CAM1_Y
                seta1 = ypr1[0][2]
                xc1 = (corners1[target1][0][0][0] + corners1[target1][0][1][0] + corners1[target1][0][2][0] + corners1[target1][0][3][0]) / 4
                yc1 = (corners1[target1][0][0][1] + corners1[target1][0][1][1] + corners1[target1][0][2][1] + corners1[target1][0][3][1]) / 4
                d1 = (abs(xc1-CAM1_PX)**2+abs(yc1-CAM1_PY)**2)**0.5
                

                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM) 
                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]
                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4
                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                

                target5 = np.where(ids5 == i)
                target5 = target5[0][0]
                aruco.drawAxis(image5, cameraMatrix, distCoeffs, rvecs5[target5], tvecs5[target5], 0.06)
                cv2.Rodrigues(rvecs5[target5], rotM, jacobian = 0)
                ypr5 = cv2.RQDecomp3x3(rotM)
                #idr is robot's id
                idr = i
                x5 = tvecs5[target5][0][0] * 100 + CAM5_X
                y5 = tvecs5[target5][0][1] * 100 + CAM5_Y
                seta5 = ypr5[0][2]
                xc5 = (corners5[target5][0][0][0] + corners5[target5][0][1][0] + corners5[target5][0][2][0] + corners5[target5][0][3][0]) / 4
                yc5 = (corners5[target5][0][0][1] + corners5[target5][0][1][1] + corners5[target5][0][2][1] + corners5[target5][0][3][1]) / 4
                d5 = (abs(xc5-CAM5_PX)**2+abs(yc5-CAM5_PY)**2)**0.5
                

                w0 = d0/(d0+d1+d3+d5)
                w1 = d1/(d0+d1+d3+d5)
                w3 = d3/(d0+d1+d3+d5)
                w5 = d5/(d0+d1+d3+d5)

                x_f = w0*x0 + w1*x1 + w3*x3 + w5*x5
                y_f = w0*y0 + w1*y1 + w3*y3 + w5*y5
                seta = w0*seta0 + w1*seta1 + w3*seta3 + w5*seta5
                #print(idr, x_f,y_f)
                al[idr]=[x_f,y_f,seta]
            # updated -----------------------------------------------------------------
            #section 15 - case : 0,3,4 cam
            elif i in ids0 and i in ids3 and i in ids4 and i not in ids5 and i not in ids2 and i not in ids1:
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )
                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i
                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]
                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4
                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5
                

                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM) 
                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]
                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4
                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                

                target4 = np.where(ids4 == i)
                target4 = target4[0][0]
                aruco.drawAxis(image4, cameraMatrix, distCoeffs, rvecs4[target4], tvecs4[target4], 0.06 )
                cv2.Rodrigues(rvecs4[target4], rotM, jacobian = 0)
                ypr4 = cv2.RQDecomp3x3(rotM)
                x4=tvecs4[target4][0][0] * 100 + CAM4_X
                y4=tvecs4[target4][0][1] * 100 + CAM4_Y
                seta4 = ypr4[0][2]
                xc4 = (corners4[target4][0][0][0] + corners4[target4][0][1][0] + corners4[target4][0][2][0] + corners4[target4][0][3][0]) / 4
                yc4 = (corners4[target4][0][0][1] + corners4[target4][0][1][1] + corners4[target4][0][2][1] + corners4[target4][0][3][1]) / 4
                d4 = (abs(xc4-CAM4_PX)**2+abs(yc4-CAM4_PY)**2)**0.5


                w0 = d0/(d0+d3+d4)
                w3 = d3/(d0+d3+d4)
                w4 = d4/(d0+d3+d4)


                x_f = w0*x0 + w3*x3 + w4*x4
                y_f = w0*y0 + w3*y3 + w4*y4
                seta = w0*seta0 + w3*seta3 + w4*seta4

                #print(idr, x_f,y_f)
                al[idr]=[x_f,y_f,seta]
            #section 15 - case : 0,2,3 cam
            elif i in ids0 and i in ids2 and i in ids3 and i not in ids5 and i not in ids4 and i not in ids1:
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )
                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i
                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]
                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4
                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5
                

                target2 = np.where(ids2 == i)
                target2 = target2[0][0]
                aruco.drawAxis(image2, cameraMatrix2, distCoeffs2, rvecs2[target2], tvecs2[target2], 0.06 )
                cv2.Rodrigues(rvecs2[target2], rotM, jacobian = 0)
                ypr2 = cv2.RQDecomp3x3(rotM)
                idr = i
                x2=tvecs2[target2][0][0] * 100 + CAM2_X
                y2=tvecs2[target2][0][1] * 100 + CAM2_Y
                seta2 = ypr2[0][2]
                xc2 = (corners2[target2][0][0][0] + corners2[target2][0][1][0] + corners2[target2][0][2][0] + corners2[target2][0][3][0]) / 4
                yc2 = (corners2[target2][0][0][1] + corners2[target2][0][1][1] + corners2[target2][0][2][1] + corners2[target2][0][3][1]) / 4
                d2 = (abs(xc2-CAM2_PX)**2+abs(yc2-CAM2_PY)**2)**0.5
                

                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM) 
                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]
                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4
                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                



                x_f = w0*x0 + w2*x2 + w3*x3
                y_f = w0*y0 + w2*y2 + w3*y3
                seta = w0*seta0 + w2*seta2 + w3*seta3

                #print(idr, x_f,y_f)
                al[idr]=[x_f,y_f,seta]

            #section 15 - case : 2,3,4 cam
            elif i in ids2 and i in ids3 and i in ids4 and i not in ids0 and i not in ids1 and i not in ids5:
                target2 = np.where(ids2 == i)
                target2 = target2[0][0]
                aruco.drawAxis(image2, cameraMatrix2, distCoeffs2, rvecs2[target2], tvecs2[target2], 0.06 )
                cv2.Rodrigues(rvecs2[target2], rotM, jacobian = 0)
                ypr2 = cv2.RQDecomp3x3(rotM)
                idr = i
                x2=tvecs2[target2][0][0] * 100 + CAM2_X
                y2=tvecs2[target2][0][1] * 100 + CAM2_Y
                seta2 = ypr2[0][2]
                xc2 = (corners2[target2][0][0][0] + corners2[target2][0][1][0] + corners2[target2][0][2][0] + corners2[target2][0][3][0]) / 4
                yc2 = (corners2[target2][0][0][1] + corners2[target2][0][1][1] + corners2[target2][0][2][1] + corners2[target2][0][3][1]) / 4
                d2 = (abs(xc2-CAM2_PX)**2+abs(yc2-CAM2_PY)**2)**0.5
                

                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM) 
                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]
                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4
                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                

                w0 = d0/(d0+d1+d5)
                w1 = d1/(d0+d1+d5)
                w5 = d5/(d0+d1+d5)


                x_f = w0*x0 + w1*x1 + w5*x5
                y_f = w0*y0 + w1*y1 + w5*y5
                seta = w0*seta0 + w1*seta1 + w5*seta5
                al[idr]=[x_f,y_f,seta]

            #section 15 - case : 0,2,4 cam
            elif i in ids0 and i in ids2 and i in ids4 and i not in ids1 and i not in ids3 and i not in ids5:
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )
                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i
                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]
                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4
                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5
                

                target2 = np.where(ids2 == i)
                target2 = target2[0][0]
                aruco.drawAxis(image2, cameraMatrix2, distCoeffs2, rvecs2[target2], tvecs2[target2], 0.06 )
                cv2.Rodrigues(rvecs2[target2], rotM, jacobian = 0)
                ypr2 = cv2.RQDecomp3x3(rotM)
                idr = i
                x2=tvecs2[target2][0][0] * 100 + CAM2_X
                y2=tvecs2[target2][0][1] * 100 + CAM2_Y
                seta2 = ypr2[0][2]
                xc2 = (corners2[target2][0][0][0] + corners2[target2][0][1][0] + corners2[target2][0][2][0] + corners2[target2][0][3][0]) / 4
                yc2 = (corners2[target2][0][0][1] + corners2[target2][0][1][1] + corners2[target2][0][2][1] + corners2[target2][0][3][1]) / 4
                d2 = (abs(xc2-CAM2_PX)**2+abs(yc2-CAM2_PY)**2)**0.5
                

                target4 = np.where(ids4 == i)
                target4 = target4[0][0]
                aruco.drawAxis(image4, cameraMatrix, distCoeffs, rvecs4[target4], tvecs4[target4], 0.06 )
                cv2.Rodrigues(rvecs4[target4], rotM, jacobian = 0)
                ypr4 = cv2.RQDecomp3x3(rotM)
                x4=tvecs4[target4][0][0] * 100 + CAM4_X
                y4=tvecs4[target4][0][1] * 100 + CAM4_Y
                seta4 = ypr4[0][2]
                xc4 = (corners4[target4][0][0][0] + corners4[target4][0][1][0] + corners4[target4][0][2][0] + corners4[target4][0][3][0]) / 4
                yc4 = (corners4[target4][0][0][1] + corners4[target4][0][1][1] + corners4[target4][0][2][1] + corners4[target4][0][3][1]) / 4
                d4 = (abs(xc4-CAM4_PX)**2+abs(yc4-CAM4_PY)**2)**0.5

                w0 = d0/(d0+d2+d4)
                w2 = d2/(d0+d2+d4)
                w4 = d4/(d0+d2+d4)


                x_f = w0*x0 + w2*x2 + w4*x4
                y_f = w0*y0 + w2*y2 + w4*y4
                seta = w0*seta0 + w2*seta2 + w4*seta4
                al[idr]=[x_f,y_f,seta]
            #section 15 - case : 0,2
            elif i in ids0 and i in ids2 and i not in ids4 and i not in ids1 and i not in ids3 and i not in ids5:
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )
                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i
                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]
                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4
                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5
                

                target2 = np.where(ids2 == i)
                target2 = target2[0][0]
                aruco.drawAxis(image2, cameraMatrix2, distCoeffs2, rvecs2[target2], tvecs2[target2], 0.06 )
                cv2.Rodrigues(rvecs2[target2], rotM, jacobian = 0)
                ypr2 = cv2.RQDecomp3x3(rotM)
                idr = i
                x2=tvecs2[target2][0][0] * 100 + CAM2_X
                y2=tvecs2[target2][0][1] * 100 + CAM2_Y
                seta2 = ypr2[0][2]
                xc2 = (corners2[target2][0][0][0] + corners2[target2][0][1][0] + corners2[target2][0][2][0] + corners2[target2][0][3][0]) / 4
                yc2 = (corners2[target2][0][0][1] + corners2[target2][0][1][1] + corners2[target2][0][2][1] + corners2[target2][0][3][1]) / 4
                d2 = (abs(xc2-CAM2_PX)**2+abs(yc2-CAM2_PY)**2)**0.5
                


                w0 = d0/(d0+d2)
                w2 = d2/(d0+d2)


                x_f = w0*x0 + w2*x2
                y_f = w0*y0 + w2*y2
                seta = w0*seta0 + w2*seta2
                al[idr]=[x_f,y_f,seta]

            #section 15 - case : 3,4
            elif i in ids3 and i in ids4 and i not in ids0 and i not in ids1 and i not in ids2 and i not in ids5:

                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM) 
                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]
                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4
                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                

                target4 = np.where(ids4 == i)
                target4 = target4[0][0]
                aruco.drawAxis(image4, cameraMatrix, distCoeffs, rvecs4[target4], tvecs4[target4], 0.06 )
                cv2.Rodrigues(rvecs4[target4], rotM, jacobian = 0)
                ypr4 = cv2.RQDecomp3x3(rotM)
                x4=tvecs4[target4][0][0] * 100 + CAM4_X
                y4=tvecs4[target4][0][1] * 100 + CAM4_Y
                seta4 = ypr4[0][2]
                xc4 = (corners4[target4][0][0][0] + corners4[target4][0][1][0] + corners4[target4][0][2][0] + corners4[target4][0][3][0]) / 4
                yc4 = (corners4[target4][0][0][1] + corners4[target4][0][1][1] + corners4[target4][0][2][1] + corners4[target4][0][3][1]) / 4
                d4 = (abs(xc4-CAM4_PX)**2+abs(yc4-CAM4_PY)**2)**0.5

                w3 = d3/(d3+d4)
                w4 = d4/(d3+d4)


                x_f = w3*x3 + w4*x4
                y_f = w3*y3 + w4*y4
                seta = w3*seta3 + w4*seta4
                al[idr]=[x_f,y_f,seta]

            #section 14 - case : 0,1,5 cam
            elif i in ids0 and i in ids1 and i in ids5 and i not in ids2 and i not in ids3 and i not in ids4:
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )
                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i
                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]
                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4
                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5

                target1 = np.where(ids1 == i)
                target1 = target1[0][0]
                aruco.drawAxis(image1, cameraMatrix, distCoeffs, rvecs1[target1], tvecs1[target1], 0.06 )
                cv2.Rodrigues(rvecs1[target1], rotM, jacobian = 0)
                ypr1 = cv2.RQDecomp3x3(rotM)
                idr = i
                x1=tvecs1[target1][0][0] * 100 + CAM1_X
                y1=tvecs1[target1][0][1] * 100 + CAM1_Y
                seta1 = ypr1[0][2]
                xc1 = (corners1[target1][0][0][0] + corners1[target1][0][1][0] + corners1[target1][0][2][0] + corners1[target1][0][3][0]) / 4
                yc1 = (corners1[target1][0][0][1] + corners1[target1][0][1][1] + corners1[target1][0][2][1] + corners1[target1][0][3][1]) / 4
                d1 = (abs(xc1-CAM1_PX)**2+abs(yc1-CAM1_PY)**2)**0.5
                

                target5 = np.where(ids5 == i)
                target5 = target5[0][0]
                aruco.drawAxis(image5, cameraMatrix, distCoeffs, rvecs5[target5], tvecs5[target5], 0.06)
                cv2.Rodrigues(rvecs5[target5], rotM, jacobian = 0)
                ypr5 = cv2.RQDecomp3x3(rotM)
                #idr is robot's id
                idr = i
                x5 = tvecs5[target5][0][0] * 100 + CAM5_X
                y5 = tvecs5[target5][0][1] * 100 + CAM5_Y
                seta5 = ypr5[0][2]
                xc5 = (corners5[target5][0][0][0] + corners5[target5][0][1][0] + corners5[target5][0][2][0] + corners5[target5][0][3][0]) / 4
                yc5 = (corners5[target5][0][0][1] + corners5[target5][0][1][1] + corners5[target5][0][2][1] + corners5[target5][0][3][1]) / 4
                d5 = (abs(xc5-CAM5_PX)**2+abs(yc5-CAM5_PY)**2)**0.5
                

                w0 = d0/(d0+d1+d5)
                w1 = d1/(d0+d1+d5)
                w5 = d5/(d0+d1+d5)


                x_f = w0*x0 + w1*x1 + w5*x5
                y_f = w0*y0 + w1*y1 + w5*y5
                seta = w0*seta0 + w1*seta1 + w5*seta5
                al[idr]=[x_f,y_f,seta]

            #section 14 - case : 1,3,5 cam
            elif i in ids1 and i in ids3 and i in ids5 and i not in ids2 and i not in ids4 and i not in ids0:
                target1 = np.where(ids1 == i)
                target1 = target1[0][0]
                aruco.drawAxis(image1, cameraMatrix, distCoeffs, rvecs1[target1], tvecs1[target1], 0.06 )
                cv2.Rodrigues(rvecs1[target1], rotM, jacobian = 0)
                ypr1 = cv2.RQDecomp3x3(rotM)
                idr = i
                x1=tvecs1[target1][0][0] * 100 + CAM1_X
                y1=tvecs1[target1][0][1] * 100 + CAM1_Y
                seta1 = ypr1[0][2]
                xc1 = (corners1[target1][0][0][0] + corners1[target1][0][1][0] + corners1[target1][0][2][0] + corners1[target1][0][3][0]) / 4
                yc1 = (corners1[target1][0][0][1] + corners1[target1][0][1][1] + corners1[target1][0][2][1] + corners1[target1][0][3][1]) / 4
                d1 = (abs(xc1-CAM1_PX)**2+abs(yc1-CAM1_PY)**2)**0.5
                

                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM) 
                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]
                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4
                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                

                target5 = np.where(ids5 == i)
                target5 = target5[0][0]
                aruco.drawAxis(image5, cameraMatrix, distCoeffs, rvecs5[target5], tvecs5[target5], 0.06)
                cv2.Rodrigues(rvecs5[target5], rotM, jacobian = 0)
                ypr5 = cv2.RQDecomp3x3(rotM)
                #idr is robot's id
                idr = i
                x5 = tvecs5[target5][0][0] * 100 + CAM5_X
                y5 = tvecs5[target5][0][1] * 100 + CAM5_Y
                seta5 = ypr5[0][2]
                xc5 = (corners5[target5][0][0][0] + corners5[target5][0][1][0] + corners5[target5][0][2][0] + corners5[target5][0][3][0]) / 4
                yc5 = (corners5[target5][0][0][1] + corners5[target5][0][1][1] + corners5[target5][0][2][1] + corners5[target5][0][3][1]) / 4
                d5 = (abs(xc5-CAM5_PX)**2+abs(yc5-CAM5_PY)**2)**0.5
                

                w1 = d1/(d1+d3+d5)
                w3 = d3/(d1+d3+d5)
                w5 = d5/(d1+d3+d5)


                x_f = w1*x1 + w3*x3 + w5*x5
                y_f = w1*y1 + w3*y3 + w5*y5
                seta = w1*seta1 + w3*seta3 + w5*seta5
                al[idr]=[x_f,y_f,seta]
            #section 14 - case : 0,1,3 cam
            elif i in ids0 and i in ids1 and i in ids3 and i not in ids2 and i not in ids4 and i not in ids5:
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )
                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i
                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]
                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4
                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5

                target1 = np.where(ids1 == i)
                target1 = target1[0][0]
                aruco.drawAxis(image1, cameraMatrix, distCoeffs, rvecs1[target1], tvecs1[target1], 0.06 )
                cv2.Rodrigues(rvecs1[target1], rotM, jacobian = 0)
                ypr1 = cv2.RQDecomp3x3(rotM)
                idr = i
                x1=tvecs1[target1][0][0] * 100 + CAM1_X
                y1=tvecs1[target1][0][1] * 100 + CAM1_Y
                seta1 = ypr1[0][2]
                xc1 = (corners1[target1][0][0][0] + corners1[target1][0][1][0] + corners1[target1][0][2][0] + corners1[target1][0][3][0]) / 4
                yc1 = (corners1[target1][0][0][1] + corners1[target1][0][1][1] + corners1[target1][0][2][1] + corners1[target1][0][3][1]) / 4
                d1 = (abs(xc1-CAM1_PX)**2+abs(yc1-CAM1_PY)**2)**0.5
                

                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM) 
                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]
                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4
                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                

                w0 = d0/(d0+d1+d3)
                w1 = d1/(d0+d1+d3)
                w3 = d3/(d0+d1+d3)


                x_f = w0*x0 + w1*x1 + w3*x3
                y_f = w0*y0 + w1*y1 + w3*y3
                seta = w0*seta0 + w1*seta1 + w3*seta3
                al[idr]=[x_f,y_f,seta]
            #section 14 - case : 0,3,5 cam
            elif i in ids0 and i in ids3 and i in ids5 and i not in ids1 and i not in ids2 and i not in ids4:
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )
                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i
                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]
                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4
                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5


                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM) 
                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]
                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4
                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                


                target5 = np.where(ids5 == i)
                target5 = target5[0][0]
                aruco.drawAxis(image5, cameraMatrix, distCoeffs, rvecs5[target5], tvecs5[target5], 0.06)
                cv2.Rodrigues(rvecs5[target5], rotM, jacobian = 0)
                ypr5 = cv2.RQDecomp3x3(rotM)
                #idr is robot's id
                idr = i
                x5 = tvecs5[target5][0][0] * 100 + CAM5_X
                y5 = tvecs5[target5][0][1] * 100 + CAM5_Y
                seta5 = ypr5[0][2]
                xc5 = (corners5[target5][0][0][0] + corners5[target5][0][1][0] + corners5[target5][0][2][0] + corners5[target5][0][3][0]) / 4
                yc5 = (corners5[target5][0][0][1] + corners5[target5][0][1][1] + corners5[target5][0][2][1] + corners5[target5][0][3][1]) / 4
                d5 = (abs(xc5-CAM5_PX)**2+abs(yc5-CAM5_PY)**2)**0.5
                


                w0 = d0/(d0+d3+d5)
                w3 = d3/(d0+d3+d5)
                w5 = d5/(d0+d3+d5)


                x_f = w0*x0 + w3*x3 + w5*x5
                y_f = w0*y0 + w3*y3 + w5*y5
                seta = w0*seta0 + w3*seta3 + w5*seta5
                al[idr]=[x_f,y_f,seta]

            #section 14 - case : 1,3
            elif i in ids1 and i in ids3 and i not in ids0 and i not in ids2 and i not in ids4 and i not in ids5:
                target1 = np.where(ids1 == i)
                target1 = target1[0][0]
                aruco.drawAxis(image1, cameraMatrix, distCoeffs, rvecs1[target1], tvecs1[target1], 0.06 )
                cv2.Rodrigues(rvecs1[target1], rotM, jacobian = 0)
                ypr1 = cv2.RQDecomp3x3(rotM)
                idr = i
                x1=tvecs1[target1][0][0] * 100 + CAM1_X
                y1=tvecs1[target1][0][1] * 100 + CAM1_Y
                seta1 = ypr1[0][2]
                xc1 = (corners1[target1][0][0][0] + corners1[target1][0][1][0] + corners1[target1][0][2][0] + corners1[target1][0][3][0]) / 4
                yc1 = (corners1[target1][0][0][1] + corners1[target1][0][1][1] + corners1[target1][0][2][1] + corners1[target1][0][3][1]) / 4
                d1 = (abs(xc1-CAM1_PX)**2+abs(yc1-CAM1_PY)**2)**0.5
                

                target3 = np.where(ids3 == i)
                target3 = target3[0][0]
                aruco.drawAxis(image3, cameraMatrix, distCoeffs, rvecs3[target3], tvecs3[target3], 0.06 )
                cv2.Rodrigues(rvecs3[target3], rotM, jacobian = 0)
                ypr3 = cv2.RQDecomp3x3(rotM) 
                x3=tvecs3[target3][0][0] * 100 + CAM3_X
                y3=tvecs3[target3][0][1] * 100 + CAM3_Y
                seta3 = ypr3[0][2]
                xc3 = (corners3[target3][0][0][0] + corners3[target3][0][1][0] + corners3[target3][0][2][0] + corners3[target3][0][3][0]) / 4
                yc3 = (corners3[target3][0][0][1] + corners3[target3][0][1][1] + corners3[target3][0][2][1] + corners3[target3][0][3][1]) / 4
                d3 = (abs(xc3-CAM3_PX)**2+abs(yc3-CAM3_PY)**2)**0.5
                


                w1 = d1/(d1+d3)
                w3 = d3/(d1+d3)



                x_f = w1*x1 + w3*x3
                y_f = w1*y1 + w3*y3
                seta = w1*seta1 + w3*seta3
                al[idr]=[x_f,y_f,seta]

            #section 14 - case : 0,5
            elif i in ids0 and i in ids5 and i not in ids1 and i not in ids2 and i not in ids3 and i not in ids4:
                target0 = np.where(ids0 == i)
                target0 = target0[0][0]
                aruco.drawAxis(image0, cameraMatrix, distCoeffs, rvecs0[target0], tvecs0[target0], 0.06 )
                cv2.Rodrigues(rvecs0[target0], rotM, jacobian = 0)
                ypr0 = cv2.RQDecomp3x3(rotM)
                idr = i
                x0=tvecs0[target0][0][0] * 100 + CAM0_X
                y0=tvecs0[target0][0][1] * 100 + CAM0_Y
                seta0 = ypr0[0][2]
                xc0 = (corners0[target0][0][0][0] + corners0[target0][0][1][0] + corners0[target0][0][2][0] + corners0[target0][0][3][0]) / 4
                yc0 = (corners0[target0][0][0][1] + corners0[target0][0][1][1] + corners0[target0][0][2][1] + corners0[target0][0][3][1]) / 4
                d0 = (abs(xc0-CAM0_PX)**2+abs(yc0-CAM0_PY)**2)**0.5

                target5 = np.where(ids5 == i)
                target5 = target5[0][0]
                aruco.drawAxis(image5, cameraMatrix, distCoeffs, rvecs5[target5], tvecs5[target5], 0.06)
                cv2.Rodrigues(rvecs5[target5], rotM, jacobian = 0)
                ypr5 = cv2.RQDecomp3x3(rotM)
                #idr is robot's id
                idr = i
                x5 = tvecs5[target5][0][0] * 100 + CAM5_X
                y5 = tvecs5[target5][0][1] * 100 + CAM5_Y
                seta5 = ypr5[0][2]
                xc5 = (corners5[target5][0][0][0] + corners5[target5][0][1][0] + corners5[target5][0][2][0] + corners5[target5][0][3][0]) / 4
                yc5 = (corners5[target5][0][0][1] + corners5[target5][0][1][1] + corners5[target5][0][2][1] + corners5[target5][0][3][1]) / 4
                d5 = (abs(xc5-CAM5_PX)**2+abs(yc5-CAM5_PY)**2)**0.5
                

                w0 = d0/(d0+d5)
                w5 = d5/(d0+d5)



                x_f = w0*x0 + w5*x5
                y_f = w0*y0 + w5*y5
                seta = w0*seta0 + w5*seta5
                al[idr]=[x_f,y_f,seta]


        
        for idr in al:
            if al[idr] != []:

                robot_id = idr
                robot_x = al[idr][0]
                robot_y = al[idr][1]
                robot_theta = al[idr][2]
                print(robot_id, robot_x, robot_y, robot_theta)
                

                
        cv2.imshow('frame',image0)
        cv2.imshow('frame2',image1)
        cv2.imshow('frame3',image2)
        cv2.imshow('frame4',image3)
        cv2.imshow('frame5',image4)
        cv2.imshow('frame6',image5)




       


        #cv2.resizeWindow('Video Out', 1920, 1536)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    f.close()
    cap0.release()
    cap1.release()
    cap2.release()
    cap3.release()
    cap4.release()
    cap5.release()
    cv2.destroyAllWindows()

1 -60.07069626479282 90.1283422435277 -177.24932226563172
2 -29.51488876241614 89.0489264227011 0.22337067789880055
3 0.7032895554486389 89.41306867174502 1.032432711336521
4 30.469708586810558 89.7344488080025 -0.3553259367768782
5 59.74601163789585 88.21455766345761 3.2188862009348127
6 90.33133369736875 89.41674452259461 1.70980136689679
7 -90.4699552477722 59.24180308010055 1.9718400353037007
8 -61.03345885990569 60.60218888562527 0.8456206233234733
9 -29.705388526434596 58.804002907343715 1.5193563803554817
10 0.33707190170266843 59.79038661811849 4.730243039236787
11 30.74206888205177 60.08958126613827 -0.2638891179990941
12 59.33532362366584 59.31633851893103 1.6225808738025445
13 89.83737060036002 60.034132104090375 1.8422031178371538
14 -89.4842825457633 29.743453944811904 0.9021914091053886
15 -59.636034386446546 30.05522097363557 1.9130434711753772
16 -29.350788886206576 29.623548992503792 1.8049361706792344
17 0.19201326309470124 30.560122038979806 -1.3166531369894523
18 30

0 -90.56946568328486 89.03156614836243 1.5579421926890833
1 -60.07069626479282 90.1283422435277 -177.24932226563172
2 -29.48651139423331 89.04986479471397 0.2745200634636327
3 0.6477903781275858 89.41469948619576 1.1013689390320769
4 30.360190784527447 89.67453098764197 -0.31772312914167034
5 59.74601163789585 88.21455766345761 3.2188862009348127
6 90.33133369736875 89.41674452259461 1.70980136689679
7 -90.23197165873776 59.36758871846101 -0.1594998811333803
8 -59.76580078248044 59.491161897144124 3.0276805746751934
9 -29.705388526434596 58.804002907343715 1.5193563803554817
10 0.3585274688408534 59.8523315218939 4.464884679575259
11 30.536166192525567 59.895757359567554 -0.3611596864717666
12 59.13904138240483 59.246257716962944 1.8953476590138312
13 89.56479175369694 60.634018938573256 0.7666996336597243
14 -89.4842825457633 29.743453944811904 0.9021914091053886
15 -59.394594062738435 30.581777657515307 -2.470091692033356
16 -29.391187125615826 29.648360471776243 2.5596411868977285
1

0 -91.04842739114233 89.0385595172727 2.199683257598726
1 -60.07069626479282 90.1283422435277 -177.24932226563172
2 -29.48651139423331 89.04986479471397 0.2745200634636327
3 0.7032895554486389 89.41306867174502 1.032432711336521
4 30.095350499204976 89.7220294219864 -0.1070520521181989
5 59.74601163789585 88.21455766345761 3.2188862009348127
6 90.33133369736875 89.41674452259461 1.70980136689679
7 -89.90005413922148 59.51031627710306 1.358991384239278
8 -60.05641400639731 59.80871379273863 3.228286581540293
9 -29.951768910373996 59.07498901211859 1.616605217694623
10 0.2539597553537348 60.0359570902429 4.314504357012453
11 30.303158371520894 60.040870057022964 -0.9324659473348694
12 59.163539543414075 59.221763968077425 1.9197391244439805
13 90.24328295251205 59.94753826735786 1.2696360837829817
14 -89.4842825457633 29.743453944811904 0.9021914091053886
15 -59.70066082620016 30.181226196355123 2.1062089275658606
16 -29.391187125615826 29.648360471776243 2.5596411868977285
17 0.17914966

0 -90.56946568328486 89.03156614836243 1.5579421926890833
1 -60.07069626479282 90.1283422435277 -177.24932226563172
2 -29.500211752611882 89.055731799499 0.27137379692707364
3 0.7032895554486389 89.41306867174502 1.032432711336521
4 29.96645619406609 89.73897796382933 -0.9930092947836549
5 59.10882452791304 88.18174774683902 3.4245581390875226
6 90.33133369736875 89.41674452259461 1.70980136689679
7 -90.23197165873776 59.36758871846101 -0.1594998811333803
8 -60.05438761316067 59.76114568376455 3.0834477749778086
9 -29.639207755497775 58.947400562612984 0.17920241586781338
10 0.2539597553537348 60.0359570902429 4.314504357012453
11 30.536166192525567 59.895757359567554 -0.3611596864717666
12 59.163539543414075 59.221763968077425 1.9197391244439805
13 90.24328295251205 59.94753826735786 1.2696360837829817
14 -89.4842825457633 29.743453944811904 0.9021914091053886
15 -59.78660774189858 30.038590176365283 2.102319579405366
16 -29.350788886206576 29.623548992503792 1.8049361706792344
17 0.3

0 -90.56946568328486 89.03156614836243 1.5579421926890833
1 -60.07069626479282 90.1283422435277 -177.24932226563172
2 -29.48651139423331 89.04986479471397 0.2745200634636327
3 0.7032895554486389 89.41306867174502 1.032432711336521
4 29.91640757953987 89.74078840434788 -0.9772646711214562
5 59.74601163789585 88.21455766345761 3.2188862009348127
6 90.33133369736875 89.41674452259461 1.70980136689679
7 -89.90005413922148 59.51031627710306 1.358991384239278
8 -59.55280062340479 59.66537822318298 2.868813902984745
9 -29.705388526434596 58.804002907343715 1.5193563803554817
10 0.23249999171469807 59.974018290401055 4.579860443875195
11 30.542499484744745 60.29635706638892 0.9692488399901666
12 59.163539543414075 59.221763968077425 1.9197391244439805
13 90.24328295251205 59.94753826735786 1.2696360837829817
14 -89.44064033354516 29.766201705200935 1.1021979169833984
15 -59.394594062738435 30.581777657515307 -2.470091692033356
16 -29.391187125615826 29.648360471776243 2.5596411868977285
17 0.3

...

0 -89.91477758983677 89.07880850852527 1.5635656882183195
1 -60.07069626479282 90.1283422435277 -177.24932226563172
2 -29.500211752611882 89.055731799499 0.27137379692707364
3 0.7032895554486389 89.41306867174502 1.032432711336521
4 30.448329562995497 89.72159356483395 -0.8172248693720858
5 59.74601163789585 88.21455766345761 3.2188862009348127
6 90.33133369736875 89.41674452259461 1.70980136689679
7 -90.23197165873776 59.36758871846101 -0.1594998811333803
8 -59.669636216695935 59.45428163425273 3.264308793210938
9 -29.957947252055313 59.00701777303538 0.41185552736164877
10 0.33516339180083193 60.23499751511581 4.659844832483738
11 30.72991606525987 60.069392406330394 -0.12738674949612716
12 59.23208514513995 59.281000764655516 2.0410380908765164
13 90.24328295251205 59.94753826735786 1.2696360837829817
14 -89.4842825457633 29.743453944811904 0.9021914091053886
15 -59.43364811331436 29.99404415705557 2.270896117238176
16 -29.331681659802825 29.630839275575394 1.7648144712940923
17 0.4

0 -89.91477758983677 89.07880850852527 1.5635656882183195
1 -60.21581671134955 90.07421867745741 -176.41882034412592
2 -29.48651139423331 89.04986479471397 0.2745200634636327
3 0.7032895554486389 89.41306867174502 1.032432711336521
4 30.48882616450902 89.66661728588615 -0.3344109886116514
5 59.74601163789585 88.21455766345761 3.2188862009348127
6 90.33133369736875 89.41674452259461 1.70980136689679
7 -89.90005413922148 59.51031627710306 1.358991384239278
8 -59.76580078248044 59.491161897144124 3.0276805746751934
9 -30.024164002624445 58.86361417820067 1.752140819444207
10 0.2539597553537348 60.0359570902429 4.314504357012453
11 30.5101563510416 60.22356300713354 -0.9375945354290733
12 59.43813648429989 59.3557262207774 1.7579773807259045
13 89.56479175369694 60.634018938573256 0.7666996336597243
14 -89.4842825457633 29.743453944811904 0.9021914091053886
15 -59.100239495817775 30.508744729172072 -3.3111634261015164
16 -29.391187125615826 29.648360471776243 2.5596411868977285
17 0.300419

0 -91.04842739114233 89.0385595172727 2.199683257598726
1 -60.07069626479282 90.1283422435277 -177.24932226563172
2 -29.48651139423331 89.04986479471397 0.2745200634636327
3 0.7032895554486389 89.41306867174502 1.032432711336521
4 30.360190784527447 89.67453098764197 -0.31772312914167034
5 59.74601163789585 88.21455766345761 3.2188862009348127
6 90.02090890253953 89.40594193957389 1.74660449258857
7 -90.23197165873776 59.36758871846101 -0.1594998811333803
8 -59.76580078248044 59.491161897144124 3.0276805746751934
9 -29.70839555077729 58.795782766668125 1.870888844726991
10 0.2539597553537348 60.0359570902429 4.314504357012453
11 30.52290730891482 59.86792452552929 0.06698202918480606
12 59.163539543414075 59.221763968077425 1.9197391244439805
13 90.01871040441132 59.779967757738234 1.0620322882368158
14 -89.44064033354516 29.766201705200935 1.1021979169833984
15 -59.727898978972604 30.067240257239902 3.1102077559093386
16 -29.35896280035285 29.62004844578322 1.799459094179265
17 0.2675

0 -91.04842739114233 89.0385595172727 2.199683257598726
1 -60.21581671134955 90.07421867745741 -176.41882034412592
2 -29.48651139423331 89.04986479471397 0.2745200634636327
3 0.7032895554486389 89.41306867174502 1.032432711336521
4 30.360190784527447 89.67453098764197 -0.31772312914167034
5 59.74601163789585 88.21455766345761 3.2188862009348127
6 90.33133369736875 89.41674452259461 1.70980136689679
7 -90.23197165873776 59.36758871846101 -0.1594998811333803
8 -59.76580078248044 59.491161897144124 3.0276805746751934
9 -29.957947252055313 59.00701777303538 0.41185552736164877
10 0.23249999171469807 59.974018290401055 4.579860443875195
11 30.526634472237518 60.28961129526708 0.9826322924324813
12 59.163539543414075 59.221763968077425 1.9197391244439805
13 90.1822505113651 59.88822299043096 1.2021836836050044
14 -89.4842825457633 29.743453944811904 0.9021914091053886
15 -59.66484958882901 30.04199858230001 1.6929065964213421
16 -29.38593631792502 29.62539148208787 2.1962660967762986
17 0.54

AttributeError: 'builtin_function_or_method' object has no attribute 'close'